#1.0. Introdução

Este arquivo contém um algorítimo responsável por realizar o pré-processamento de textos utilizando a biblioteca de processamento de linguagem natural Spacy. Esse tratamento textual será utilizado para formatar os textos buscados a partir de um WebCrawler, reorganizá-lo de modo a simplificar o entendimento dos assuntos abordados nesses textos e posteriormente alimentar a base de dados utilizada pelo assistente virtual Nexus. Sendo assim, espera-se que ao final de todo o processamento, os textos utilizados para alimentar a base de dados exprimam de maneira clara os principasi tópicos mais relvantes para aquele conteúdo e facilite o processo de busca por parte do Nexus.

#2.0. Importação de dependências

Primeiramente, é necessário realizar a instalação da biblioteca Spacy, responsável por fornecer funções que auxiliam no processamento dos textos por meio do uso de algumas ferramentas comumente utilizadas para o processamento de linguagem natural (PLN), como a geração de tokens (Tokenizing) e a remoção de stopwords. Dessa forma, na célula de código abaixo é possível visualizar a importação dessa biblioteca bem como outros recusros advindos dela que também serão importantes para o PLN.
<br><br>
Além disso, também foi importada a biblioteca Pandas e Numpy. O Pandas será utilizado para auxiliar na visualização dos dados tratados ao final do processo por meio de um dataframe gerado com o auxílio dessa dependência. Já o Numpy será utilizado para
<br><br>
A importação dessas bibliotecas e suas dependências pode ser vizualisada nas células de código abaixo:

In [42]:
!pip install spacy
!pip install spacy-lang-pt


ERROR: Could not find a version that satisfies the requirement spacy-lang-pt (from versions: none)
ERROR: No matching distribution found for spacy-lang-pt


In [2]:
import spacy
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np

In [3]:


spacy.cli.download('pt_core_news_lg')
nlp = spacy.load("pt_core_news_lg")



✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [4]:
stop_words = nlp.Defaults.stop_words
#stop_words

#3.0. Criando as Funções

A seguir, é possível visualizar as funções criadas a partir das bibliotecas instaladas para auxiliar no processamento de linguagem natural. Cada função foi modularizada de modo a obter uma melhor separação de funções entre cada uma delas e facilitar a testagem de suas funcionalidades posteriormente

## 3.1. toLowerCase( )

O primeiro passo é padronizar os texto recebidos para garantir o mínimo de variações não relevantes para o entendimento do algorítmo quanto ao significado do conteúdo do texto. Sendo assim, é necessário que todos os caracteres estejam escritos em letra minúscula para evitar que a mesma palavra escrita em minúsculo ou maiúsculo seja considerada duas palavras diferentes pelo algorítimo. Para isso, foi criada a função `toLowerCase()`, que recebe como parâmetro um texto em formato *.txt* e percorre todo esse texto substituindo as letras maiúsculas por minúsculas. Essa substituição ocorre por meio do método nativo da linguagem de programação Python `.lower()`. A execução desse processo pode ser visualizada na célula de código abaixo:

In [5]:
def toLowerCase(_news):
    return _news.lower()

##3.2. removeStopWords( )

Outro fator que pode comprometer o entendimento do significado textual por parte do algorítimo é a presença de stopwords. Essas são palavras utilizadas somente para conectar estruturas sintáticas, sem apresentar de fato um significado relevante para o texto e portanto não devem ser levadas em consideração durante o aprendizado do algorítimo.
<br><br>
A biblioteca Spacy possúi por padrão um agrupamento de palavras consideradas stopwords, sendo em sua maioria artigos, preposições e conjunções. Dessa forma, é criado uma estrutura de repetição dentro da função `removeStopWords()` que percorres cada uma das palavras passadas como parâmetro e as compara com as stopwords pré-definidas. Se essas palavras representarem uma stopword, o algorítmo irá desconsiderá-lo e ele não entrará na lista de palavras filtradas nesse processo, que pode ser visualizado na célula de código abaixo:

In [6]:
def removeStopWords(text):
    doc = nlp(text)

    tokens = [token.text for token in doc if not token.is_stop]
    cleaned_text = ' '.join(tokens)

    return cleaned_text


##3.3. newsTreatment( )

Com o intuito de agrupar as funções criadas anteriormente e garantir uma linha de código mais sinteizada, foi criada a função `newsTreatMent()`, que recebe como parâmetro uma lista de notícias obtidas pelo Web Crawler e aplica sobre elas as funções `toLowerCase()` e `removeStopWords()`. Essa função pode ser visualizada na célula de código abaixo:

In [7]:
def newsTreatment(list_of_news):
    new_list = []
    for reportage in list_of_news:
        new_list.append(removeStopWords(toLowerCase(reportage)))

    return new_list

##3.4. turnToVector( )

O próximo passo a se fazer é separar cada uma das palavras contidas em um texto como tokens, visto que inicialmente todas as palavras lidas pelo algorítmo são uma única *string* sem qualquer conexão ou lógica entre os textos contidos dentro dela.
<br><br>
Em seguida, é necessário transformar todos os tokens de um único texto em vetores baseados na frequência em que cada um deles aparece ao longo do texto. Assim, posteriormente, será possível comparar a similaridade desses vetores e propor respostas com base nessas comparações.
<br><br>
Para isso foi criada a função `turnToVector()`, que recebe como parâmetro uma lista de textos e tokeniza cada uma das palavras conitdas neles por meio da função importada `Tokenizer()`. Em seguida, usa outro método importado chamado `.text_to_matrix()` para selecionadar as palavras presentes em cada um dos textos e criar vetores com base na frequência em que aprecem em cada um deles. Essa função pode ser visualizada na célula de código abaixo:

In [8]:
def turnToVector(_list):

    ## Step 1: Determine the Vocabulary
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(_list)
    print(f'Vocabulary: {list(tokenizer.word_index.keys())}')

    ## Step 2: Count
    vectors = tokenizer.texts_to_matrix(_list, mode='count')

    return vectors

##3.5. get_similarity

A fim de compararmos os textos obtidos e facilitar o aprendizado do algoritimo por meio da similaridade dos vetores que represetam esses textos, foi criada a função `get_similarity()`, que recebe como parâmetro uma lista de vetores e um valor numérico correspondete ao índice desses vetores dentro da lista. Em seguida, é realizado o cálculo de similaridade do cosseno entre entre esses dois vetores e é retornado um valor que varia de 0 a 1, sendo 0 textos completamentes distintos e 1 textos completamente idênticos. A função mencionada pode ser visualizada na célula de código abaixo:

In [9]:
def get_similarity(_list,a,b):
    vetor1 = np.array(_list[a])
    vetor2 = np.array(_list[b])

    produto_escalar = np.dot(vetor1, vetor2)
    norma_vetor1 = np.linalg.norm(vetor1)
    norma_vetor2 = np.linalg.norm(vetor2)

    similaridade = produto_escalar / (norma_vetor1 * norma_vetor2)

    return similaridade


##3.6. pipeLine( )

A fim de sintetizar todas as funções de pré-processamento criadas anteriormente, foi criada uma última função chamada `pipeLine()`. Essa função recebe como parâmetro uma lista de textos em formato *.txt* e submete essa lista aos métodos criados anteriormente. Em seguida, ela retorna uma tabela que contém um comparativo entre cada texto em sua forma original e sua forma tratada, facilitando a transparência do que foi realizado. A função criada pode ser visualizada na célula de código abaixo:

In [10]:
def pipeLine(_list):
    preProcess = newsTreatment(_list)
    vectorization = turnToVector(preProcess)

    array = np.array([[_list[0],preProcess[0], vectorization[0] ],
                  [ _list[1],preProcess[1], vectorization[1]],
                  [ _list[2],preProcess[2], vectorization[2]],
                  [ _list[3],preProcess[3], vectorization[3]],
                  [ _list[4],preProcess[4], vectorization[4]],
                  [ _list[5],preProcess[5], vectorization[5]],
                  [ _list[6],preProcess[6], vectorization[6]],
                  [ _list[7],preProcess[7], vectorization[7]],
                  [ _list[8],preProcess[8], vectorization[8]],
                  [ _list[9],preProcess[9], vectorization[9]]
                  ])

    nomes_colunas = ['Sentença', 'Sentença filtrada', 'Vetor']
    nomes_linhas = ['Frase 1', 'Frase 2','Frase 3', 'Frase 4','Frase 5','Frase 6','Frase 7','Frase 8','Frase 9','Frase 10']

    df = pd.DataFrame(array, columns=nomes_colunas, index=nomes_linhas)

    return df


#4.0. Realização de Testes

Com a finalidade de garantir um melhor desempenho para o algorítmo e validar os processos realizados acima, foram realizados alguns testes unitários para averiguar o pleno funcionamento das funções descritas acima.
<br><br>
Cada teste possui como nome o prefixo *test* acrescentado ao nome da função a ser testada.
<br><br>
A verificação de cada teste pode ser vizualizada nas células de código abaixo:

##4.1. Teste Função toLowerCase

In [11]:
def testLowerCase():
    originalText = "Esse é Um Exemplo de Frase"
    answer = toLowerCase(originalText)

    if answer == "esse é um exemplo de frase":
        return True
    else:
        return False

In [12]:
testLowerCase()

True

##4.2. Teste Função removeStopWords

In [13]:
def testRemoveStopWords():
    originalText2 ="esse é um exemplo de frase"
    answer2 = removeStopWords(originalText2)

    if answer2 == "frase":
        return True
    else:
        return False


In [14]:
testRemoveStopWords()

True

##4.3. Teste newsTreatment

In [15]:
def testNewsTreatment():
    originalText3 = ["Essa é a frase 1","Essa é a frase 2","Essa é a frase 3","Essa é a frase 4","Essa é a frase 5"]
    answer3 = newsTreatment(originalText3)

    if answer3 == ['frase 1','frase 2','frase 3','frase 4','frase 5']:
        return True
    else:
        return False


In [16]:
testNewsTreatment()

True

##4.4. Teste turnToVector

In [17]:
def testTurnToVector():
    originalText4 = ["Essa é a frase 1","Essa é a frase 2"]
    answer4 = turnToVector(originalText4)
    rightAnswer1 = [0., 1., 1., 1., 1., 1., 0.]
    rightAnswer2 = [0., 1., 1., 1., 1., 0., 1.]
    rightAnswer = []
    rightAnswer.append(rightAnswer1)
    rightAnswer.append(rightAnswer2)


    for i  in range(len(answer4)):
        if  set(answer4[i]) != set(rightAnswer[i]):
            return False
        else:
                continue
    return True

In [18]:
testTurnToVector()

Vocabulary: ['essa', 'é', 'a', 'frase', '1', '2']


True

##4.5. Teste pipeLine

In [19]:
def testPipeLine():
    originalText5 = ["Essa é a frase 1","Essa é a frase 2","Essa é a frase 3","Essa é a frase 4","Essa é a frase 5","Essa é a frase 6","Essa é a frase 7","Essa é a frase 8","Essa é a frase 9","Essa é a frase 10"]
    answer5 = pipeLine(originalText5)

    print(type(answer5))

    if type(answer5) == "<class 'pandas.core.frame.DataFrame'>":
        return True
    else:
        return False

In [21]:
testPipeLine()

Vocabulary: ['frase', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
<class 'pandas.core.frame.DataFrame'>


<ipython-input-10-58f0e796c2cf>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.array([[_list[0],preProcess[0], vectorization[0] ],


False

#5.0. Tratamento de dados

A fim de demonstrar e compreender o funcionamento do algoritimo criado, foi realizado um *case* de exemplo no qual uma série de notícias retiradas do google serão submetidas ao pré processamento descrito em cada uma das funções e no final espera-se obter um comparativo entre essas noticias.


##5.1. Definição dos textos a serem utilizados

Primeiramente, o conteúdo de cada uma das 10 noticias foi armazenado em uma variável.

In [22]:
news1= """Acesse nossa plataforma a maior comunidade de TI do Brasil

    Fóruns
    Notícias
    IT Forum Series
    Colunas
    Estudos
    Vídeos
    Podcasts
    Revistas

HIT

Materiais educativos e gratuitos

    iforum icon social Anuncie
    iforum icon social Envie seu Release
    iforum icon social Fale conosco
    iforum icon social Revistas
    iforum icon social Newsletter

Logo IT
IT Forum é um produto do grupo IT Mídia saiba mais
IT Forum
Email
Realizado por:

    Facebook
    linkedin
    twitter
    telegram
    whatsapp
    msg

    Home > Notícias > Computerworld Brasil > Bradesco amplia parceria com IBM...

Bradesco amplia parceria com IBM para explorar criptografia pós-quântica
Projeto piloto utilizará tecnologia da IBM que buscará por criptográficos e vulnerabilidades no Open Finance do banco
Redação
7:58 pm - 01 de agosto de 2023
Bradesco
Imagem: Shutterstock

De olho no futuro da computação quântica, o Bradesco, por meio do seu ecossistema de inovação inovabra, anunciou que ampliará seu relacionamento com a IBM para explorar e avaliar o uso de criptografia quântica segura. Inicialmente, será utilizada a aplicação do Open Finance do banco como projeto piloto para implementar o IBM Quantum Safe.

A tecnologia da IBM varre e localiza criptográficos, pendências e vulnerabilidades. Isso permitirá ao Bradesco criar um Inventário de Materiais Criptográficos (CBOM) para melhor planejar, implementar e aplicar na prática a tecnologia de criptografia pós-quântica.

O Bradesco lembra que o setor financeiro já utiliza normalmente a criptografia para transações bancárias, certificados digitais, armazenamento dos dados, entre outros. Agora, os especialistas, por meio do IBM Quantum Safe Explorer, ajudarão o banco a identificar potenciais algoritmos criptográficos vulneráveis nos seus sistemas, auxiliando na identificação de aplicações sob risco de ataques usando computação quântica.

Com base nessa avaliação, o Bradesco planeja criar uma metodologia para identificar outras aplicações que possam se beneficiar de tecnologias de criptografia pós-quânticas seguras e prepará-las para integrar os algoritmos resistentes a futuros computadores quânticos criptograficamente relevantes.

Leia ainda: Totvs e Itaú Unibanco finalizam criação da Techfin

“Uma das consequências da computação quântica em larga escala no futuro pode ser seu impacto nos sistemas de proteção de dados. Entendemos que nossa preparação é prioridade neste momento. Afinal, a segurança é inerente à nossa missão como banco”, afirma Cíntia Barcelos, diretora de tecnologia do Bradesco.

O Bradesco também anunciou que agora integra o IBM Quantum Network para explorar as aplicações da indústria de computação quântica. Ao aderir à esta rede, o Bradesco passa a ter acesso à expertise e aos softwares quânticos da IBM.

Com o movimento, o Bradesco busca ampliar as experimentações com a computação quântica, que vêm sendo realizadas desde 2021, incluindo experimentos em otimização, (distribuição de caixa e otimização de portfólio); forecasting (projeção de variáveis macroeconômicas) e simulação (precificação de derivativos e análise de risco)

“À medida que as capacidades da computação quântica avançam, isso pode nos permitir um dia ver uma maior eficiência operacional e a resolução de problemas que atualmente são considerados difíceis para os computadores clássicos. Esperamos que a tecnologia possa ser amplamente utilizada em um período de 5 a 10 anos”, diz Cíntia.

O banco também afirma estar investindo em treinamentos científicos para equipar seu time de TI e base de funcionários com o conhecimento necessário para iniciar o planejamento da integração da tecnologia quântica em suas operações.

Siga o IT Forum no LinkedIn e fique por dentro de todas as notícias!
Tags:
Bradesco
computação quântica
criptografia
IBM
open finance
EM ALTA NO IT FORUM
Notícias
People’s Choice: conheça os CEOs finalistas do prêmio Executivo de TI 2023
Notícias
Hackers burlam autenticação multifator e avançam contra contas de CEOs e CFOs
Notícias
Bruna Zamith revela como é ser cientista de dados na Amazon
INSCREVA-SE NA NOSSA NEWSLETTER

Notícias relacionadas
Computerworld Brasil
Conforme a adoção dos dispositivos de realidade virtual aumenta, questões de privacidade podem surgir
Notícias
TIM passa a oferecer atendimento a partir da busca do Google
Notícias
Dia da Informática: 6 empresas que marcaram a história da TI no Brasil
Notícias
CMOs apostam em IA generativa como vantagem competitiva, diz BCG
Newsletter de tecnologia para você

Os melhores conteúdos do IT Forum, ComputerWorld, CIO e CSO na sua caixa de entrada.

IT Forum

Fóruns

    Todos os Fóruns

Notícias

    Todas as Notícias
    Liderança
    Carreira
    Negócios
    Tendências
    Computerworld Brasil
    CIO
    CSO

COLUNAS

    Todas as Colunas

ESTUDOS

    Todos os Estudos

VIDEOS

    Assistir Vídeos

PODCASTS

    Escutar Podcasts

HIT

    Acessar HIT

    itforum icon social Anuncie
    itforum icon social Envie seu Release
    itforum icon social Fale conosco
    itforum icon social Revistas
    itforum icon social Newsletter

Política de privacidade
Termos de Uso
Copyright 2022 IT MIDIA - Todos os Direitos Reservados"""
#tokens

In [23]:
news2 = """Trecho de adutora estoura e 250 mil moradores ficam sem água no Grande Recife, diz Compesa
Suspensão do abastecimento afeta localidades do Cabo de Santo Agostinho e de Jaboatão dos Guararapes, desde a noite da segunda-feira (21), segundo a Compesa.
Por g1 PE

22/08/2023 16h44  Atualizado há 18 minutos

Imagem de arquivo mostra pingo de água caindo de torneira — Foto: Alex Rocha/PMPA
Imagem de arquivo mostra pingo de água caindo de torneira — Foto: Alex Rocha/PMPA


Moradores de duas cidades do Grande Recife estão sem água desde as 23h da segunda-feira (21). Ao todo, são 250 mil pessoas afetadas pela suspensão do abastecimento, segundo a Companhia Pernambucana de Saneamento (Compesa).

📲 Compartilhe no WhatsApp

As localidades afetadas foram:

Pontezinha (Cabo de Santo Agostinho);
Curcurana, Guararapes, Barra de Jangada, Candeias e Piedade (Jaboatão dos Guararapes).
O motivo foi o estouramento ocorrido em um trecho de 750 milímetros, em ferro fundido, da Adutora de Gurjau, na Avenida Conde da Boa Vista, no bairro de Pontezinha, no Cabo de Santo Agostinho, no Grande Recife.

A expectativa inicial da Compesa é que os serviços durem 24 horas, podendo o prazo se estender a depender da complexidade das intervenções.


Ainda de acordo com a Compesa, o fechamento dos registros para conter o fluxo da água é o primeiro procedimento para os serviços de reparo. Isso também permite o esvaziamento da adutora para que o conserto do problema seja efetivamente iniciado.

A Compesa detalhou como são os serviços de reparo realizados no local:

Por se tratar de uma tubulação de grande porte, técnicos mobilizaram equipamentos pesados, materiais e profissionais para recuperar esse trecho da adutora;
Após montada a estrutura, os técnicos iniciaram a escavação até alcançar o trecho do tubo danificado e, assim, fechar o diagnóstico sobre os danos causados;
O trabalho é acompanhado por técnicos de outras empresas prestadoras de serviço na área;
O órgão municipal de trânsito do Cabo de Santo Agostinho foi acionado para ajudar no desvio disciplinar do trânsito;
Foi mobilizada uma equipe social para realizar o levantamento dos danos materiais registrados na área por causa do estouramento, para as providências. """

In [24]:
news3 = """ Mizael Bispo é solto após Justiça conceder progressão para o regime aberto
O ex-policial foi condenado a mais de 20 anos de prisão pela morte da ex-namorada Mércia Nakashima em 2010. Ele cumpria pena em um presidio em Tremembé, no interior de SP.
Por g1 Vale do Paraíba e Região

22/08/2023 16h45  Atualizado há 54 segundos

Imagem de arquivo - Mizael Bispo, condenado por matar Mércia Nakashima. — Foto: Nelson Antoine/Fotoarena/Estadão Conteúdo
Imagem de arquivo - Mizael Bispo, condenado por matar Mércia Nakashima. — Foto: Nelson Antoine/Fotoarena/Estadão Conteúdo


Mizael Bispo, ex-policial condenado a mais de 20 anos de prisão pela morte da ex-namorada Mércia Nakashima, foi solto na tarde desta terça-feira (22), após a Justiça conceder progressão para o regime aberto (entenda mais abaixo). Ele cumpria pena há 13 anos e, atualmente, estava em um presídio em Tremembé, no interior de São Paulo.

Compartilhe no Whatsapp
Compartilhe no Telegram
O g1 apurou que a decisão que concedeu progressão para o regime aberto foi expedida pela 2ª Vara de Execuções Criminais de Taubaté. Com a decisão, Bispo deixou a Penitenciária Dr. José Augusto Salgado, a P2, em Tremembé, por volta das 16h.


A informação foi confirmado pelo advogado de defesa de Mizael. Ele informou que Bispo "cumpriu tudo que a lei estabeleceu e lhe foi concedido a progressão de regime aberto".

Regime aberto
No regime aberto, o condenado cumpre pena fora da prisão e pode trabalhar durante o dia. À noite, deve se recolher em endereço autorizado pela Justiça.

A legislação determina que o preso se recolha no período noturno em uma casa de albergado - modelo prisional que abriga presos que estão no mesmo regime -, mas o Estado de São Paulo não dispõe desse tipo de unidade prisional. Por isso, na prática, os presos vão para casa.

Para não perder o benefício, o condenado precisa seguir algumas regras, como:

permanecer no endereço que for designado durante o repouso e nos dias de folga;
cumprir os horários combinados para ir e voltar do trabalho;
não pode se ausentar da cidade onde reside sem autorização judicial;
quando determinado, deve comparecer em juízo, para informar e justificar suas atividades.
Mesmo seguindo essas condições básicas, o juiz pode estabelecer outras condições especiais, de acordo com cada caso.

O crime
Mizael Bispo cumpria pena de 21 anos e 3 meses de prisão no regime semiaberto na Penitenciária 2, em Tremembé, no interior de São Paulo. O benefício do regime semiaberto foi concedido pela Justiça em 2022.


Mércia estava desaparecida desde 23 de maio de 2010, quando foi vista pela última vez em Guarulhos, na Grande São Paulo. Seu carro e seu corpo foram encontrados pela Polícia Civil, respectivamente, em 10 e 11 de junho numa represa em Nazaré Paulista, na região metropolitana.

Segundo a perícia, a advogada foi baleada e morreu afogada. A vítima tinha 28 anos. A investigação e o Ministério Público acusaram Mizael de matar Mércia por ciúmes e vingança por ela não ter reatado o namoro com ele. De acordo com a acusação, um homem chamado Evandro o ajudou a fugir do local.

Mizael e Evandro foram condenados pelos crimes de homicídio doloso qualificado por motivo torpe, meio cruel e recurso que dificultou a defesa de Mércia. Em outras ocasiões, os dois sempre negaram os crimes e se disseram inocentes."""

In [25]:
news4 = """Acusados de matar jovem quando ele ia para casa da avó em Rio Branco são absolvidos
Henrique Ferreira da Silva, de 20 anos, foi morto em 2016. Decisão foi publicada nesta terça-feira (22).
Por g1 AC — Rio Branco

22/08/2023 16h44  Atualizado há 21 minutos

Acusados de matar jovem quando ele ia para casa da avó em Rio Branco são absolvidos  — Foto: Arquivo pessoal
Acusados de matar jovem quando ele ia para casa da avó em Rio Branco são absolvidos — Foto: Arquivo pessoal


O Conselho de Sentença do Tribunal do Júri absolveu Saymon Wallace Fonseca do Nascimento e Francisco Gabriel Bastos Velozo pela morte de Henrique Ferreira da Silva, de 20 anos, em 21 de dezembro de 2016. O jovem foi morto a tiros no bairro Bahia Velha, em Rio Branco, quando estava indo para a casa da avó e foi surpreendido por dois homens em uma motocicleta.

Leia também:
Jovem é executado a tiros quando ia para casa da avó em Rio Branco
"Ele estava a cerca de 10 metros da casa da avó quando foi surpreendido. Uma outra tia dele que estava na casa da avó, ouviu três disparos e tentou correr até lá, mas um primo do Henrique segurou ela. Tinham dois caras na moto, um deles desceu e atirou nas costas do meu sobrinho. Depois que minha irmã viu que eles foram embora, correu até lá e viu que era o Henrique caído no chão", contou Eliane Silva, tia da vítima, na época do crime.


No processo, Saymon Fonseca disse que não teve nenhuma participação nos fatos e que está sendo acusado porque terceiros falaram que foi o autor, mas é mentira. “Disse que o bairro estava em disputa de domínio pelas facções, que foi condenado por integrar facção. Que só ficou sabendo da morte do Henrique em 2020 quando foi acusado.”

Já Francisco Velozo também disse que não havia nenhuma participação nos fatos, que conheceu o co-réu do bairro e a vítima do bairro. "Que não sabia que a vítima colava com facção criminosa, que ficou sabendo aqui na audiência. Que não colava nem integrava facção criminosa. Que acredita que lhe envolveram neste crime porque já foi condenado por outro crime de homicídio. Que não tinha moto. Que a mãe da vítima sabe quem foi. Que escutou comentários que foram dois caras do bairro e um já morreu”, contou no processo. """

In [26]:
news5= """Guarda-vidas de Maricá resgatam pescador e cachorrinho após barco virar em Itaipuaçu
Caso aconteceu na tarde de segunda-feira (21), na altura da rua 70. Ninguém ficou ferido.
Por Bianca Chaqboudet, g1 — Maricá

22/08/2023 16h42  Atualizado há 28 minutos

Pescador foi atingido por onda e barco acabou virando em praia de Maricá, mas ele e o cachorrinho que o acompanha na embarcação foram salvos — Foto: Imagens cedida por Jimmy Lopez
Pescador foi atingido por onda e barco acabou virando em praia de Maricá, mas ele e o cachorrinho que o acompanha na embarcação foram salvos — Foto: Imagens cedida por Jimmy Lopez


Um pescador e o cachorrinho dele foram resgatados na tarde desta segunda-feira (21), em Maricá, na Região Metropolitana do Rio, depois do barco em que eles estavam virar.

O acidente aconteceu na altura da rua 70, na Praia de Itaipuaçu.

Segundo os guarda-vidas da Defesa Civil, que atuaram no local, o homem tentava entrar no mar quando foi atingido por uma grande onda lateral. Os agentes entraram na água e conseguiram retirar o senhor e o 'feijão', cachorrinho que estava no barco, sem ferimentos.

Banhistas que estavam no local se juntaram aos guarda-vidas para resgatar a embarcação, que também não sofreu danos. """

In [27]:
news6 = """Casos de meningite crescem em AL; conheça os sintomas e saiba como se prevenir
Especialista alerta que contágio por bactérias e vírus que causam a doença cresce durante o inverno. SUS oferece vacina gratuitamente nos postos de saúde para crianças, adolescentes e adultos.
Por Vivi Leão e Heliana Gonçalves, g1 AL

22/08/2023 16h40  Atualizado há 27 minutos

Meningite bacteriana — Foto: Royalty Stock Photo/Science Photo Library/Arquivo AFP
Meningite bacteriana — Foto: Royalty Stock Photo/Science Photo Library/Arquivo AFP


Considerada uma doença endêmica no Brasil, a meningite pode ser confundida com a gripe e, na sua forma grave, pode até matar. Nos seis primeiros meses deste ano, Alagoas registrou 29 casos de meningite (sendo 21 na capital) e 5 mortes, números acima do que foi registrado no mesmo período anterior, quando foram 24 casos e 4 mortes, segundo a Secretaria de Estado da Saúde.

📲 Clique aqui para se inscrever e receber no seu WhatsApp as notícias do g1

Esse aumento de casos preocupa porque é no inverno que cresce o risco de contágio e essa sazonalidade pode refletir diretamente nos números do segundo semestre. O mais recente caso, ainda em investigação, é de uma criança que faleceu no domingo (20) em Arapiraca.


"O aumento dos casos se deve a vários fatores. Primeiro a variação sazonal, que de tempos em tempos a doença ganha força, todas as doenças infecciosas têm essa característica. Tanto os vírus quanto as bactérias têm momentos que elas se sobressaem e depois diminui a intensidade. Então nós estamos num momento de maior atividade das bactérias e dos vírus que causam meningite", explicou o médico Fernando Maia, em entrevista ao g1.

LEIA TAMBÉM

Maceió registra em 6 meses mais da metade do total de casos de meningite de 2022
As crianças são a faixa etária mais atingida, principalmente as menores de 5 anos. Dependendo do tipo de meningite, em apenas algumas horas após o início dos sintomas, a pessoa infectada pode apresentar um quadro de bastante gravidade. A meningite bacteriana é responsável pelos quadros mais fatais, como explica Maia.

"A doença meningocócica é um tipo específico de meningite bacteriana e é a mais grave, porque ela pode levar à forma fulminante, a pessoa pode morrer em 24 horas após o início da doença. Ela tem essa capacidade de fazer uma doença muito grave", explicou o médico.
Compartilhe no WhatsApp
Compartilhe no Telegram
Ao contrário da meningite bacteriana, a viral é mais benigna e pode ser até tratada em casa. Mas a dúvida que paira é como distinguir as duas.


"Os sintomas da meningite geralmente são febre alta e muita dor de cabeça. Isso é comum aos dois tipos. As meningites virais costumam ser mais leves, então a dor de cabeça é menos intensa, a febre não é tão alta e o paciente se recupera mais rápido. Na meningite bacteriana geralmente dá febre muito alta, rigidez da nuca, o pescoço fica meio travado e aparecem manchas escuras na pele", afirma Maia

Transmissão e diagnóstico
A meningite é transmitida quando pequenas gotas de saliva da pessoa infectada entram em contato com as mucosas do nariz ou da boca de um indivíduo saudável, seja através do espirro ou da tosse.

Na suspeita clínica da meningite, a indicação é a coleta do líquor, líquido que banha o sistema nervoso.

Vacinando para prevenir
Somente a vacinação pode salvar uma criança contra a meningite — Foto: Divulgação/Agência Brasil
Somente a vacinação pode salvar uma criança contra a meningite — Foto: Divulgação/Agência Brasil


O SUS oferece quatro tipos de vacina contra meningites bacterianas e qualquer pessoa pode se vacinar, criança, adolescente ou adulto. As vacinas estão disponíveis em todos os postos de saúde de Maceió a qualquer época do ano, não é preciso esperar a campanha nacional (veja lista mais abaixo).

No Brasil, existem vacinas para todos os sorotipos de meningite, mas o sorotipo B está apenas no particular e custa, em média, R$ 500 a dose.

Dados da Secretaria de Saúde de Maceió (SMS) apontam que a cobertura vacinal contra a meningite no primeiro semestre do ano na capital é de 78,96%, número abaixo da média estipulada pelo Ministério da Saúde, que é de 95%. Lembrando que o esquema vacinal compreende a vacinação em 3 doses: aos 3 meses de idade, aos 5 meses e com 1 ano (dose de reforço).

"Apesar de a vacina estar disponível nos postos de saúde, a gente ainda tem muitas crianças que não estão vacinadas, ou estão com a vacinação incompleta, já que nesse caso tem que ter duas doses para completar o esquema", ressaltou Fernando Maia.

Quais vacinas o SUS oferece?
BCG: que protege contra a meningite turberculosa
Pentavalente: protege contra as infecções invasivas, entre elas a meningite causada pelo Haemophilus influenzae sorotipo b
Meningocócica C: protege contra a doença meningocócica causada pela Neisseria meningitidis sorogrupo C
Pneumocócica 10: protege contra as infecções invasivas, entre elas a meningite causada por dez sorotipos do Streptococcus pneumoniae """

In [28]:
news7 = """Jornada Delegada de policiais e bombeiros reforça a Segurança Pública em Sorriso
A jornada extra dos policiais militares e bombeiros não resulta em gastos extras ao Governo do Estado. Os valores são pagos pela Administração Municipal de Sorriso.
TOPO
Por Prefeitura de Sorriso

22/08/2023 16h39  Atualizado há 32 minutos


Para ar um reforço a mais na Segurança Pública, a Prefeitura Municipal de Sorriso firmou um Termo de Cooperação com o Governo do Estado de Mato Grosso, por meio da Secretaria de Estado de Segurança Pública para a implementação da Jornada Delegada, com o objetivo de ampliar o número de policiais e de bombeiros militares nas ruas. A iniciativa foi apresentada pelo Gabinete de Gestão Integrada (GGI), que conta com representantes de várias forças de segurança como a Polícia Militar (PM), Polícia Judiciária Civil (PJC), Centro Integrado de Operações Aéreas (Ciopaer), Polícia Rodoviária Federal (PRF), Secretaria de Segurança Municipal de Trânsito e Defesa Civil, Conselho Municipal de Segurança Pública (COMSEP), Guarda Municipal e Poder Judiciário de Sorriso.

A Jornada Delegada já é realidade para policiais militares desde 2020 e neste ano de 2023 também foi implantada junto ao Corpo de Bombeiros, por meio do Termo de Cooperação 0055/2023. De acordo com a Lei 2.201, de maio de 2013, a jornada delegada pode ser viabilizada para apoio à fiscalização do comércio ilegal e irregular; combate à depredação do patrimônio público; apoio à fiscalização ambiental, de trânsito e licenças; e combate ao fogo. Pela regra, cada beneficiado não pode executar carga horária diária superior a 6 horas, nem tampouco carga horária mensal superior a 50 horas. A jornada extra dos policiais militares e bombeiros não resulta em gastos extras ao Governo do Estado. Os valores são pagos pela Administração Municipal de Sorriso, que conta com uma previsão orçamentária de R$ 700 mil para este ano de 2023.


Para o prefeito de Sorriso, Ari Lafin, o aumento do efetivo tanto da Polícia Militar, quanto do Corpo de Bombeiros é um reforço significativo para o combate à criminalidade e no atendimento à população, no que diz respeito ao combate a queimadas e incêndios.

A Jornada Delegada já é realidade para policiais militares desde 2020 — Foto: Assessoria
A Jornada Delegada já é realidade para policiais militares desde 2020 — Foto: Assessoria

O secretário Municipal de Segurança Pública, Trânsito e Defesa Civil, José Carlos Moura diz que a Prefeitura vem investindo recursos para auxiliar todas as forças de segurança do município “Estamos nos empenhando para dar todo aporte necessário, obviamente, que dentro da legalidade. Estamos investindo na Jornada Delegada, tanto para policiais militares, quanto para o Corpo de Bombeiros. Os militares que estão de folga, trabalham para o município e recebem o vencimento referente ao trabalho prestado, mensalmente, diretamente em suas contas-correntes. E isso tem gerado excelentes resultados”.

Para o comandante da 10.ª CIBM de Sorriso, capitão Daniel Alves Moura e Silva, a Jornada Delegada representa um incentivo a mais para os integrantes da corporação. “Nos sentimos muito felizes em poder contribuir nas horas de folga e, com isso, receber um complemento de renda, por meio desta parceria com a Prefeitura. Diariamente são quatro militares de serviço, com a Jornada Delegada, conseguimos escalar mais dois militares por dia, ou seja, temos um aumento de 50% no efetivo”.

A jornada extra dos policiais militares e bombeiros não resulta em gastos extras ao Governo do Estado. — Foto: Assessoria
A jornada extra dos policiais militares e bombeiros não resulta em gastos extras ao Governo do Estado. — Foto: Assessoria

De acordo com o comandante do 12º Batalhão da Polícia Militar de Sorriso, tenente-coronel Jorge Almeida, a Polícia Militar tem recebido investimentos tanto da Secretaria de Estado de Segurança Pública, quanto da Prefeitura de Sorriso, por meio de um termo de cooperação técnica para a Jornada Delegada, por meio da qual o município paga o policial militar no seu momento de folga que prestam serviço em ações que o município executa. “Por meio da Jornada Delegada é possível ampliar o quatro de policiais que atuam no dia a dia nas ruas da cidade e nos distritos de Primavera do Norte (Primaverinha), Caravággio e Boa Esperança para proporcionar maior sensação de segurança para nossa população. O município emprega esse valor para que o policial de folga, sirva e proteja nossa sociedade”.

Além disso, de acordo com o secretário José Carlos Moura, a Prefeitura ajuda na manutenção das forças de segurança para melhoria do atendimento à população, por meio do COMSEP (Conselho Municipal de Segurança Pública). “Temos um orçamento de aproximadamente 2 milhões de reais do FUNSEP (Fundo Municipal de Segurança Pública) para dar apoio às forças de segurança, no que se refere a mobílias, manutenção de ar-condicionado, material de limpeza, material de expediente, cedência de servidores do município, manutenção de prédios, que muitas vezes pode demorar um pouco e o município faz esse repasse para buscar sanar essas necessidades e para melhor atender as famílias sorrisenses”."""

In [29]:
news8 = """ Criança de 2 anos é picada na pálpebra por escorpião enquanto dormia no interior de SP
Caso ocorreu no dia 14 de agosto, mas o g1 teve acesso à informação nesta terça-feira (22). Depois de receber atendimento médico, o menino foi liberado.
Por g1 Rio Preto e Araçatuba

22/08/2023 16h36  Atualizado há 37 minutos

Escorpião picou pálpebra de criança em Rio Preto (SP) — Foto: Renata da Costa dos Reis/Arquivo pessoal
Escorpião picou pálpebra de criança em Rio Preto (SP) — Foto: Renata da Costa dos Reis/Arquivo pessoal


Uma criança de dois anos foi socorrida após ser picada por um escorpião enquanto dormia em São José do Rio Preto (SP). O caso ocorreu no dia 14 de agosto, mas a informação só foi divulgada nesta terça-feira (22).

Conforme apurado pelo g1, a mãe acordou com o choro da criança, por volta das 4h, e, quando chegou ao quarto, viu o escorpião na parede.

Hospital da Criança e Maternidade (HCM) de São José do Rio Preto (SP) — Foto: Foto: Divulgação
Hospital da Criança e Maternidade (HCM) de São José do Rio Preto (SP) — Foto: Foto: Divulgação


O menino foi picado na pálpebra. Ele foi socorrido e levado para a Unidade de Pronto Atendimento (UPA) e depois encaminhado ao Hospital da Criança e Maternidade (HCM).

Depois de receber atendimento médico, a criança foi liberada e não teve a visão prejudicada. Conforme a assessoria do HCM, não foi necessária a aplicação do soro antiescorpiônico."""

In [31]:
news9 = """ Inscrições para aulas de espanhol gratuitas na Fatec acabam nesta sexta-feira, em Presidente Prudente
Ao todo, são 40 vagas para interessados a partir dos 16 anos, que participarão das aulas presenciais aos sábados, das 9h às 10h30.
Por g1 Presidente Prudente

22/08/2023 16h47  Atualizado há 27 minutos

Inscrições para aulas de espanhol gratuitas na Fatec acabam nesta sexta-feira, em Presidente Prudente (SP) — Foto: Cedida/Secom
Inscrições para aulas de espanhol gratuitas na Fatec acabam nesta sexta-feira, em Presidente Prudente (SP) — Foto: Cedida/Secom


Seguem até sexta-feira (25) as inscrições para o curso gratuito de língua espanhola oferecido pela Faculdade de Tecnologia do Estado de São Paulo (Fatec), em Presidente Prudente (SP). Podem participar jovens e adultos a partir dos 16 anos.

A iniciativa, promovida em parceria com a Coordenadoria Municipal da Juventude, oferece 40 vagas para os interessados, que devem preencher um formulário on-line. As vagas serão oferecidas por ordem de inscrição.

O curso tem como objetivo auxiliar o desenvolvimento de competências exigidas pelo mercado de trabalho e em processos seletivos ou para quem deseja viajar e enriquecer o currículo, por meio de conteúdos de nível básico, como conversação, compreensão auditiva, escrita e leitura.


As aulas presenciais serão ministradas na sede da Fatec, em Presidente Prudente, a partir deste sábado (26), das 9h às 10h30, pela professora de espanhol da unidade, Silmara Moscatelli.

Serviço
A Fatec de Presidente Prudente está localizada na Rua Teresina, nº 75, na Vila Paulo Roberto. Para mais informações, acesse o site."""

In [32]:
news10 = """PF abre inquérito para investigar se houve sabotagem em apagão elétrico no país
A pedido do governo, Polícia Federal vai investigar razões que levaram à interrupção do fornecimento de energia em todas as regiões brasileiras; investigação vai apurar se houve falha técnica ou humana

Macapá no escuro após apagão que afetou estado do Amapá
Macapá no escuro após apagão que afetou estado do Amapá
Foto: Maksuel Martins/Fotoarena/Estadão Conteúdo (19.nov.2020)

Basília Rodrigues

22/08/2023 às 17:05
Compartilhe:
Facebook
Twitter
Linkedin
WhatsApp
flipboard
Ouvir notícia

A Polícia Federal abriu inquérito nesta terça-feira (22) para investigar as causas do apagão de energia elétrica que atingiu 25 estados e o Distrito Federal, na semana passada. A investigação seguirá em sigilo e vai apurar os crimes de sabotagem e atentado contra a segurança de serviço de utilidade pública.

A PF atendeu a um pedido feito pelos ministérios da Justiça e de Minas e Energia. A investigação ficará a cargo da Diretoria de Inteligência Policial da PF (DIP), em Brasília.

Vídeo: Falha que provocou apagão deve ter ocorrido por excesso de energia no sistema, dizem fontes


Apagão: Falha deve ter ocorrido por excesso de energia no sistema, dizem fontes | CNN 360ºApagão: Falha deve ter ocorrido por excesso de energia no sistema, dizem fontes | CNN 360º
A partir do inquérito, a polícia poderá ouvir pessoas do setor elétrico e do governo. Uma sobrecarga em linha operacionalizada pela Eletrobras, no Ceará, foi identificada como um dos motivos para o apagão.

Entre as hipóteses, há desde a possibilidade de uma falha técnica à ação humana, incluindo a modalidade dolosa, quando há intenção de cometer o crime.

De acordo com o governo federal, o ocorrido foi “extremamente raro”. A falha no Sistema Interligado Nacional foi registrada por volta de 8h30. Foi um fato que causou interrupção nas regiões Norte e Nordeste.

Leia mais

Há sobra de energia no país e apagão foi “erro técnico”, diz Rui Costa
Há sobra de energia no país e apagão foi “erro técnico”, diz Rui Costa
Apagão e reajuste ampliam artilharia política contra ministro de Minas e Energia, dizem fontes
Apagão e reajuste ampliam artilharia política contra ministro de Minas e Energia, dizem fontes
Após apagão, Janja faz post citando a privatização da Eletrobras
Após apagão, Janja faz post citando a privatização da Eletrobras
O apagão se espalhou pelo país devido a uma ação preventiva de proteção do sistema, em que o Operador Nacional do Sistema (ONS) minimizou a carga das regiões Sul, Sudeste e Centro-Oeste, para que não houvesse a interrupção total dessas regiões.

O apagão prejudicou cerca de 29 milhões de residências e estabelecimentos comerciais. Em alguns lugares, a falta de luz durou dez minutos, em outros, mais de seis horas.

Tópicos
Tópicos """

Em seguida, essas variáveis foram agrupadas em uma lista para facilitar a manipulação de todas elas

In [33]:
newsList = [news1,news2,news3,news4,news5,news6,news7,news8,news9,news10]

o valor final dessa lista pode ser visualizado a seguir:

In [34]:
newsTreatment(newsList)

['acesse plataforma comunidade ti brasil \n\n     fóruns \n     notícias \n     it forum series \n     colunas \n     estudos \n     vídeos \n     podcasts \n     revistas \n\n hit \n\n materiais educativos gratuitos \n\n     iforum icon social anuncie \n     iforum icon social envie release \n     iforum icon social fale conosco \n     iforum icon social revistas \n     iforum icon social newsletter \n\n it \n it forum produto it mídia saiba \n it forum \n email \n realizado : \n\n     facebook \n     linkedin \n     twitter \n     telegram \n     whatsapp \n     msg \n\n     home > notícias > computerworld brasil > bradesco amplia parceria ibm ... \n\n bradesco amplia parceria ibm explorar criptografia pós-quântica \n projeto piloto utilizará tecnologia ibm buscará criptográficos vulnerabilidades open finance banco \n redação \n 7:58 pm - 01 agosto 2023 \n bradesco \n imagem : shutterstock \n\n olho futuro computação quântica , bradesco , ecossistema inovação inovabra , anunciou ampl

##5.2. Submetendo as noticias à pipeline de pré processamento

Em seguida, todas essas notícias serão submetidas à função `pipeLine()`, que abriga as demais funções e realiza todo o pré-processamento dos textos passados como parâmetros.

In [43]:
treatedNews = pipeLine(newsList)

Vocabulary: ['foto', '—', 'meningite', 'segurança', 'jornada', 'apagão', '2023', '22', 'notícias', 'anos', 'feira', 'há', 'rio', 'criança', 'bradesco', 'g1', 'militares', 'social', 'quântica', '”', 'municipal', 'delegada', 'policiais', 'it', 'icon', 'ibm', '“', 'arquivo', 'sp', 'casa', 'polícia', 'sorriso', 'energia', 'brasil', '08', 'regime', 'bombeiros', 'pública', 'tecnologia', 'dia', '21', 'atualizado', 'minutos', 'mizael', 'condenado', 'governo', 'forum', 'computação', 'horas', 'acordo', 'bispo', 'mércia', 'matar', 'avó', 'doença', 'município', 'presidente', 'prudente', 'whatsapp', 'criptografia', 'atendimento', 'água', 'compesa', 'compartilhe', 'bairro', 'aberto', 'paulo', 'josé', 'caso', 'branco', 'casos', 'saúde', 'bacteriana', 'ano', 'prefeitura', 'preto', 'fatec', 'falha', 'iforum', 'banco', 'imagem', '10', 'ia', 'itforum', 'ocorrido', 'serviço', 'trânsito', 'justiça', 'policial', '20', 'terça', 'defesa', 'folga', 'crime', 'civil', 'vítima', 'investigação', 'jovem', 'henrique

<ipython-input-10-58f0e796c2cf>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.array([[_list[0],preProcess[0], vectorization[0] ],


Como próximo passo, é chamada a variável que contém o dataframe responsável por exibir as comparações entre o texto original e sua versão vetorizada após todo o tratamento

In [44]:
treatedNews

,Sentença,Sentença filtrada,Vetor
Frase 1,Acesse nossa plataforma a maior comunidade de ...,acesse plataforma comunidade ti brasil \n\n ...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 0.0, ..."
Frase 2,Trecho de adutora estoura e 250 mil moradores ...,trecho adutora estoura 250 moradores ficam águ...,"[0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
Frase 3,Mizael Bispo é solto após Justiça conceder pr...,mizael bispo solto justiça conceder progress...,"[0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, ..."
Frase 4,Acusados de matar jovem quando ele ia para cas...,acusados matar jovem ia casa avó rio branco ab...,"[0.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, ..."
Frase 5,Guarda-vidas de Maricá resgatam pescador e cac...,guarda-vidas maricá resgatam pescador cachorri...,"[0.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
Frase 6,Casos de meningite crescem em AL; conheça os s...,casos meningite crescem al ; conheça sintomas ...,"[0.0, 4.0, 4.0, 22.0, 0.0, 0.0, 0.0, 1.0, 1.0,..."
Frase 7,Jornada Delegada de policiais e bombeiros refo...,jornada delegada policiais bombeiros reforça s...,"[0.0, 4.0, 4.0, 0.0, 14.0, 15.0, 0.0, 4.0, 1.0..."
Frase 8,Criança de 2 anos é picada na pálpebra por es...,criança 2 anos picada pálpebra escorpião dor...,"[0.0, 6.0, 4.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, ..."
Frase 9,Inscrições para aulas de espanhol gratuitas n...,inscrições aulas espanhol gratuitas fatec ac...,"[0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
Frase 10,PF abre inquérito para investigar se houve sab...,pf abre inquérito investigar houve sabotagem a...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 15.0, 1.0, 2.0,..."


##5.3. Comparando as notícias

Por fim, é chamada a função `get_similarity()` para comparar as notícias de índice 0 e 1, ou seja, as duas primeiras notícias que aparecem na lista *newsList*

In [47]:
get_similarity(treatedNews['Vetor'],0,1)

0.024907894331711502

Como pode ser visualizado acima, após todo o pré-processamento das notícias e suas transformações em vetores, a similaridade da primeira noticia com a segunda coresponde a um valor de aproximadamente 0,2 ; o que pode ser considerado um valor bem baixo, mas sobretudo esperado visto que ambas as noticias de fato não possuem nenhum assunto em comum, sendo essa ínfima semelhança resultado de possíveis jargões jornalísticos presentes em cada um dos textos

#6.0. Conclusão

Ao fim do processo, pode-se concluir que o algorítimo possui um desempenho que satisfaz as expectativas esperadas com sua implementação, sendo capaz de tratar, tokenizar, vetorizar e comparar uma sequência de textos passadas a ele. Além disso, todos os teste de cada função validam seu correto funcionamento. Sendo assim, em um primeiro momento, constata-se que o algorítmo desenvolvido é completamente capaz de realizar o Processamento de Linguagem Natural (PLN), e deverá ser implemtendado em futuras entregas junto ao restante da aplicação